In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
import pyttsx3
import threading

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [8]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [11]:
draw_landmarks(frame, results)

In [2]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'plt' is not defined

In [13]:
len(results.left_hand_landmarks.landmark)

21

In [14]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [15]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [16]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [17]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [18]:
result_test = extract_keypoints(results)

In [19]:
result_test

array([ 0.51505268,  0.40182006, -1.00719237, ...,  0.        ,
        0.        ,  0.        ])

In [20]:
np.save('0', result_test)

In [21]:
np.load('0.npy')

array([ 0.51505268,  0.40182006, -1.00719237, ...,  0.        ,
        0.        ,  0.        ])

In [24]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
ASL=['namaste', 'noaction']
actions = np.array(ASL)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0


In [25]:

for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [26]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [27]:
cap.release()
cv2.destroyAllWindows()

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [29]:
label_map = {label:num for num, label in enumerate(actions)}

In [30]:
label_map

{'namaste': 0, 'noaction': 1}

In [31]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [32]:
np.array(sequences).shape

(60, 30, 1662)

In [33]:
res.shape

(1662,)

In [34]:
np.array(labels).shape

(60,)

In [35]:
X = np.array(sequences)

In [36]:
X.shape

(60, 30, 1662)

In [37]:
y = to_categorical(labels).astype(int)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
y_test.shape

(12, 2)

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [42]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [43]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [45]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[tb_callback])

Epoch 1/20
2/2 [==============================] - ETA: 0s - loss: 0.6815 - categorical_accuracy: 0.5625

2/2 [==============================] - 2s 1s/step - loss: 0.6815 - categorical_accuracy: 0.5625 - val_loss: 0.6449 - val_categorical_accuracy: 0.6667
Epoch 2/20
2/2 [==============================] - 1s 946ms/step - loss: 0.6618 - categorical_accuracy: 0.5625 - val_loss: 0.6030 - val_categorical_accuracy: 0.6667
Epoch 3/20
2/2 [==============================] - 1s 766ms/step - loss: 0.6357 - categorical_accuracy: 0.5625 - val_loss: 0.5189 - val_categorical_accuracy: 0.6667
Epoch 4/20
2/2 [==============================] - 1s 581ms/step - loss: 0.6070 - categorical_accuracy: 0.4792 - val_loss: 0.4539 - val_categorical_accuracy: 0.8333
Epoch 5/20
2/2 [==============================] - 1s 732ms/step - loss: 0.5560 - categorical_accuracy: 0.6042 - val_loss: 0.3867 - val_categorical_accuracy: 0.9167
Epoch 6/20
2/2 [==============================] - 1s 462ms/step - loss: 0.4657 - categorical_accuracy: 0.7708 - val_loss: 0.3184 - val_categorical_accuracy: 0.9167
Epoch 7/20
2/2 [==============

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           916992    
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 30, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8

In [47]:
res = model.predict(X_test)

1/1 [==============================] - 3s 3s/step


In [48]:
res.shape

(12, 2)

In [1]:
actions[np.argmax(res[1])]

NameError: name 'actions' is not defined

In [50]:
actions[np.argmax(y_test[1])]

'noaction'

In [53]:
model.save('action.h5')

In [45]:
#from tensorflow.keras import Model
model.load_weights('action.h5')

In [46]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [47]:
yhat = model.predict(X_test)

4/4 [==============================] - 0s 75ms/step


In [48]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
# Define your true labels and predicted labels
y_true = ytrue # your true labels
y_pred = yhat # your predicted labels

# Get the confusion matrices
cm = multilabel_confusion_matrix(y_true, y_pred)

# Print the classification report with metrics
print(classification_report(y_true, y_pred))

# Plot the confusion matrices with colors
classes = ['hello','namaste','i love you', 'thanks', 'noaction', 'what', 'where', 'scissors', 'what time', 'salute', 'pen', 'phone', 'money', 'clothes', 'good', 'bad', 'all the best','choking', 'headache', 'salam'] # your class labels

for i, cls in enumerate(classes):
    fig, axs = plt.subplots(figsize=(8,8))
    cm_i = cm[i]
    cm_i_arr = np.array([[cm_i[1][1], cm_i[0][1]], [cm_i[1][0], cm_i[0][0]]])
    axs.imshow(cm_i_arr, cmap='Blues')
    axs.set_xticks([0, 1])
    axs.set_yticks([0, 1])
    axs.set_xticklabels([cls+'-', cls+'+'])
    axs.set_yticklabels([cls+'-', cls+'+'])
    axs.set_xlabel('Predicted')
    axs.set_ylabel('True')
    axs.set_title('Confusion Matrix for '+cls)
    
    plt.tight_layout()
    plt.show()


In [51]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# assuming ytrue and yhat are lists of predictions and true labels
ytrue = np.array(ytrue)
yhat = np.array(yhat)

# Compute confusion matrix
cm = confusion_matrix(ytrue.argmax(axis=1), yhat.argmax(axis=1))
# Normalize confusion matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot heatmap
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

fmt = '.2f'  # Format for displaying values in heatmap
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize

# y_true is the true labels, and yhat is the predicted labels
# convert the labels to binary format using one-hot encoding
y_true_binarized = label_binarize(ytrue, classes=list(range(20)))
yhat_binarized = label_binarize(yhat, classes=list(range(20)))

# compute the precision and recall for each class separately
precision = dict()
recall = dict()
for i in range(20):
    precision[i], recall[i], _ = precision_recall_curve(y_true_binarized[:, i], yhat_binarized[:, i])

# compute the average precision and recall across all classes
avg_precision = dict()
avg_recall = dict()
for i in range(20):
    avg_precision[i] = np.average(precision[i])
    avg_recall[i] = np.average(recall[i])

# plot the precision-recall curve for each class separately
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
for i in range(20):
    plt.plot(recall[i], precision[i], label=f"Class {i}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

# plot the average precision-recall curve across all classes
plt.figure(figsize=(10,8))
plt.plot(list(avg_recall.values()), list(avg_precision.values()))
plt.xlabel("Average Recall")
plt.ylabel("Average Precision")
plt.title("Average Precision-Recall Curve")
plt.show()


In [54]:
accuracy_score(ytrue, yhat)

0.925

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Assuming ytrue and yhat are arrays of true and predicted values, respectively, with shape (num_samples, num_classes)
ytrue = np.array(ytrue)
yhat = np.array(yhat)

# Calculate MAE, RMSE, MAPE, and R-squared values for each class
mae_scores = mean_absolute_error(ytrue, yhat, multioutput='raw_values')
rmse_scores = np.sqrt(mean_squared_error(ytrue, yhat, multioutput='raw_values'))
mape_scores = np.mean(np.abs((ytrue - yhat) / ytrue), axis=0) * 100
r2_scores = r2_score(ytrue, yhat, multioutput='raw_values')

# Print the metrics for each class
for i in range(num_classes):
    print('Class {}:'.format(i))
    print('MAE: {:.4f}'.format(mae_scores[i]))
    print('RMSE: {:.4f}'.format(rmse_scores[i]))
    print('MAPE: {:.4f}%'.format(mape_scores[i]))
    print('R-squared: {:.4f}'.format(r2_scores[i]))
    print()
    
# Calculate the overall metrics for all classes
overall_mae = mean_absolute_error(ytrue, yhat)
overall_rmse = np.sqrt(mean_squared_error(ytrue, yhat))
overall_mape = np.mean(np.abs((ytrue - yhat) / ytrue)) * 100
overall_r2 = r2_score(ytrue, yhat)

# Print the overall metrics
print('Overall metrics:')
print('MAE: {:.4f}'.format(overall_mae))
print('RMSE: {:.4f}'.format(overall_rmse))
print('MAPE: {:.4f}%'.format(overall_mape))
print('R-squared: {:.4f}'.format(overall_r2))
print('Accuracy: ',accuracy_score(ytrue, yhat))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# assuming ytrue and yhat have shape (n_samples, 20)
from sklearn.preprocessing import label_binarize

ytrue = label_binarize(ytrue, classes=list(range(20)))
yhat = label_binarize(yhat, classes=list(range(20)))

n_classes = 20
fpr = [None] * n_classes
tpr = [None] * n_classes
roc_auc = [None] * n_classes

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(ytrue[:, i], yhat[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure()
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan',
          'magenta', 'yellow', 'indigo', 'lime', 'teal', 'maroon', 'navy', 'turquoise', 'violet', 'black']

for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=2, label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for multi-class')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)) # adjust legend position
plt.show()

In [56]:
from scipy import stats

In [57]:
import numpy as np
import cv2

colors = [(245,117,16), (117,245,16), (16,117,245)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        color = tuple(np.array(colors[num], dtype=np.uint8))
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), color, -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return output_frame

#plt.figure(figsize=(18,18))
#plt.imshow(prob_viz(res, actions, image, colors))


In [ ]:
import pyttsx3
import threading
sequence = []
sentence = []
predictions = []
threshold = 0.7
engine = pyttsx3.init()
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        print('len ', len(sequence))
        res = None
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predicted_sign = actions[np.argmax(res)]  # get the predicted sign
            print(predicted_sign)

            # Convert predicted sign to speech
            def speak():
                engine.say(predicted_sign)
                engine.runAndWait()  # play the speech
            thread = threading.Thread(target=speak)
            thread.start()

        #3 Viz logic
        #if res[np.argmax(res)].all() > threshold:
        if res is not None and res[np.argmax(res)].all() > threshold:

            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
        if len(sentence) > 5: 
            sentence = sentence[-5:]
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [279]:
cap.release()
cv2.destroyAllWindows()